#Exerici 1: Creació d'individus
##Definició del genotip i fenotip
    Genotip -> Matriu de 16 files (quarks) i 28 columnes (orbital)
    Fenotip -> Comportament resultant per tant en aquest cas cal tenir en compte: 
                - Que cap orbital superi el màxim d'influències permeses.
                - Que cap quark influeixi més de dos orbitals consecutius.

#Estructura de l'ADN
La matriu és binària per representar la influència o no dels quarks sobre els orbitals.
D'aquesta manera les operaciones genètiques posteriors són senzilles i es pot avaluar ràpidament si el resultat obtingut és correcte.

#Principi de variació
La inicialització és aleatòria però controlada per tal de garantir que es compleixin les normes de no influenciar més de dos orbitals consecutius i de no sobrecarregar els orbitals.
Mitjançant mutacions amb probabilitat es canvien els 0 a 1 i els 1 a 0 comprovant que no es trenquin les restriccions. Per tant, mantenint una diversitat genètica en la població inicial i permeten variació a altres configuracions per a les generacions futures evitant l'estancament en un o una sèrie de patrons. 

In [11]:
import numpy as np
import pygame
import random
def create_adn():

    initial = np.zeros((16,28), dtype=int)
    orbitals_count = np.zeros(28, dtype=int)
    max_influence = [8,8,8,8,10,6,4] * 4

    for quark in range(16):
        for orbital in range(28):
            influence = np.random.randint(0, 2)
            initial[quark, orbital] = influence
            '''
            influence_available = True

            if orbital >= 2 and initial[quark, orbital-1] == 1 and initial[quark, orbital-2] == 1:
                influence_available = False
            if influence == 1 and orbitals_count[orbital] >= max_influence[orbital]:
                influence_available = False
            if influence == 1 and influence_available:
                orbitals_count[orbital] += 1
                initial[quark, orbital] = 1
            '''
    
    return initial

initial = create_adn()
print(initial)

[[1 0 1 1 0 1 0 0 0 0 1 1 0 1 0 1 1 1 1 0 1 1 0 1 1 1 1 0]
 [1 1 0 0 1 0 0 1 1 0 0 0 0 0 1 1 1 1 1 0 0 1 0 0 0 1 1 0]
 [1 0 0 0 1 1 1 1 1 1 0 0 0 1 0 0 1 0 0 1 1 1 0 0 0 0 0 0]
 [0 0 1 1 1 0 1 1 0 1 0 0 0 1 0 1 0 0 0 1 1 0 1 0 1 1 1 1]
 [1 1 1 1 0 0 0 1 1 0 0 1 1 0 0 1 1 0 0 0 1 0 1 0 1 0 0 0]
 [1 1 1 0 1 0 1 0 0 1 0 1 0 0 0 0 1 1 1 1 1 1 1 0 1 0 1 0]
 [0 0 0 1 1 1 0 0 0 1 1 1 1 1 1 0 0 1 0 1 1 1 1 1 0 1 1 1]
 [0 1 1 1 1 0 1 1 0 0 1 0 0 0 1 1 1 0 0 1 0 1 1 1 1 0 0 0]
 [0 0 1 0 0 0 0 0 1 1 1 0 0 1 1 1 0 1 1 1 0 0 0 1 0 1 1 1]
 [1 0 1 1 1 1 1 0 1 0 0 0 0 1 1 1 1 0 0 1 1 0 0 1 0 1 1 1]
 [0 1 0 1 0 0 1 0 0 1 1 0 1 0 1 0 1 0 1 0 0 0 1 0 1 1 1 1]
 [1 1 1 1 1 0 0 0 1 1 1 1 0 0 0 0 0 1 1 0 1 1 0 1 1 0 1 0]
 [0 1 0 1 0 1 1 0 1 1 0 1 0 1 1 0 1 1 0 0 0 1 0 1 0 0 1 1]
 [0 0 0 0 1 0 1 0 1 0 1 0 0 0 0 0 1 1 1 0 1 1 0 0 0 0 0 1]
 [1 1 0 0 1 0 1 0 0 1 1 0 1 1 0 0 0 1 1 1 0 1 0 0 0 1 0 1]
 [1 1 1 0 1 1 1 1 0 1 0 0 1 1 0 0 1 0 0 0 0 0 0 0 0 0 0 1]]


#Exercici 2: Funció Fitness
Objectiu de la funció és mesurar com de bona és la configuració del resultat. En aquest cas l'objectiu és minimitzar aquest fitness per reduir les penalitzacions i per tant aconseguir un millor individu.
Per avauluar-ho cal tenir en compte el límits d'influència per evitar sobrecàrregues als orbitals. La penalització en cas de no complir-se aquesta condició serà de +5.
D'altra banda també cal penalizar que un quark no pugui influenciar tres orbitals consecutius. En aquesta altra casuística també es penalitzarà amb +10.

In [12]:
def fitness(adn):
    penalty = 0
    max_influence = [8,8,8,8,10,6,4] * 4
    orbitals_count = np.sum(adn, axis=0)

    for i in range(28):
        if orbitals_count[i] > max_influence[i]:
            penalty += (orbitals_count[i] - max_influence[i]) * 5
    for quark in range (16):
        for orbital in range(26):
            if adn[quark, orbital] == 1 and adn[quark,orbital+1] == 1 and adn[quark, orbital+2] == 1:
                penalty += 10

    return penalty

fitness_value = fitness(initial)
print(fitness_value)

790


# Exercici 3: Creació de la matching pool
Procediment de selecció elitista:
S'agafen els millors individus i que per tant tenen el fitness més baix mentre la resta de la població es descarta.
D'aquesta manera assegurem evitar que per atzar es descartin individus bons i com que en aquest cas hi ha poca variància en les penalitzacions un mètode de ruleta seria gariebé aleatòri.


In [13]:
def elitist_selection(population, fitness_values, elite_size):
    selected_indices = np.argsort(fitness_values)
    best_indices = selected_indices[:elite_size]
    matching_pool = [population[i] for i in best_indices]
    return matching_pool

# Exercici 4: Crossover i mutació

Procediment de reproducció: Crossover
A cada quark es decideix per fila si agafar-la del pare 1 o pare 2 amb probabilitat 50%. Deixant de banda la simplicitat de la informació aquest mètode permet barrejar inforació dels dos pares de manera equilibrada i eviten descompensacions que podrien provocar perdre una estructura útil.

Respecte a la mutació després del crossover s'ha decidit afegir una probabilitat de l'1% de mutar, per tant invertir el valor. Com la matriu és de 16x28 = 446 l'1% serien entre 4 i 5 mutacions a la matriu. D'aquesta manera, s'aconsegueixen introduir petites variacions aleatòries i puntuals evintant l'estancament però sense renunciar a l'estructura que va prenent forma la matriu.

In [14]:
def crossover_5050(parent1, parent2):
    child = np.zeros_like(parent1)
    for quark in range(16):
        if np.random.rand() < 0.5:
            child[quark] = parent1[quark]
        else:
            child[quark] = parent2[quark]
    return child

def mutate(adn, mutation_rate=0.01):
    for quark in range(16):
        for orbital in range(28):
            if np.random.rand() < mutation_rate:
                adn[quark, orbital] = 1 - adn[quark, orbital]
    return adn

In [15]:
pygame.init()
class Food:
    def __init__ (self, radius, color):
        self.radius = radius
        self.pos = np.array([np.random.randint(self.radius, 800 - self.radius), np.random.randint(self.radius, 600 - self.radius)])
        self.color = color

    def reposition(self, screen):
        self.pos = np.array([np.random.randint(self.radius, screen.get_width() - self.radius), np.random.randint(self.radius, screen.get_height() - self.radius)])

    def draw(self, screen):
        pygame.draw.circle(screen, self.color, self.pos, self.radius)
class Ball:
    def __init__ (self, radius, mass, pos_initial, color):
        self.radius = radius
        self.mass = mass
        self.color = color
        self.pos = np.array(pos_initial)
        self.vel = np.array(np.zeros(2))
        self.vel = np.array([0,0])
        self.acc = np.array(np.zeros(2))
        self.gravity = np.array([0, 9.81])
        self.max_speed = 10
        self.max_steering = 1

    def apply_force(self, force):     
        self.acc = self.acc + (np.array(force) / self.mass)
   
    def update(self, dt, food):

        nearest_distance = 99999
        nearest_index = -1

        for i, f in enumerate(food):
            dist = np.linalg.norm(self.pos - f.pos)
            if (dist < nearest_distance):
                nearest_distance = dist
                nearest_index = i
    
        nearestFood = food[nearest_index]


        #Trobar direcció desitjada
        desired_direction = nearestFood.pos - self.pos
        desired_direction_norm = np.linalg.norm(desired_direction)

        #Limitar direcció màxima
        if (desired_direction_norm > self.max_speed):
            desired_direction = (desired_direction / desired_direction_norm) * self.max_speed

        #Calcular força de rotació
        steering_force_dir = desired_direction - self.vel
        self.apply_force(steering_force_dir)

        self.vel = self.vel + self.acc * dt

        vel_norm = np.linalg.norm(self.vel)
        if vel_norm > self.max_speed:
            self.vel = (self.vel / vel_norm) * self.max_steering

        self.pos = self.pos + self.vel * dt 
        self.acc = np.array([0, 0])





    def check_collision(self, screen, food, other_ball):
        #CHECK FOR FOOD COLISION
        for f in food:
            distance2food = np.linalg.norm(self.pos - f.pos)
            sum_radi = self.radius + f.radius

            if distance2food <= sum_radi:
                f.reposition(screen)
                if self.radius < 100:
                    self.radius += 0.25*f.radius


        #CHECK FOR OTHER BALL COLISION
        distance2ball = np.linalg.norm(self.pos - other_ball.pos)
        sum_radi = self.radius + other_ball.radius

        if distance2ball <= sum_radi:
            self.collision_ball(other_ball)
            if (self.radius >= other_ball.radius):
                if self.radius < 100:
                    self.radius += 0.25*other_ball.radius
                other_ball.pos = np.array([np.random.randint(self.radius, 800 - self.radius), np.random.randint(self.radius, 600 - self.radius)])
                other_ball.radius = 10


    def get_kinetic_energy(self): 
        energy = np.array([0, 0])
        energy = 0.5 * self.mass * np.pow(self.vel,2)
        return energy   

    def get_potential_energy(self, screen:pygame.Surface):
        energy = np.array([0, 0])
        energy = self.mass * self.gravity * (screen.get_height() - self.pos)
        return energy

    


    def collision_ball(self, other_ball):

        #Trobar els vectors unitaris de l'eix de xoc
        u = np.array(other_ball.pos - self.pos) / np.linalg.norm(other_ball.pos - self.pos)

        #Distància de seguretat
        if (np.abs(np.linalg.norm(other_ball.pos - self.pos)) <= (self.radius + other_ball.radius)):
            self.pos = self.pos + 1.2*(-u * ((self.radius + other_ball.radius) - np.linalg.norm(other_ball.pos - self.pos)))

        #Trobar els vectors unitaris de l'eix de xoc
        w = np.array([-u[1], u[0]])

        #Calcular angles per les projeccions
        if np.linalg.norm(self.vel) > 0 and np.linalg.norm(other_ball.pos) > 0:
            theta = np.arccos(np.dot(u, self.vel) / (np.linalg.norm(self.vel) * np.linalg.norm(u)))
            if (np.cross(u, self.vel) < 0):
                theta = -theta
        else:
            theta = 0

        if (np.linalg.norm(other_ball.vel) > 0) and np.linalg.norm(other_ball.pos) > 0:
            phi = np.arccos(np.dot(u, other_ball.vel) / (np.linalg.norm(other_ball.vel) * np.linalg.norm(u)))
            if (np.cross(u, other_ball.vel) < 0):
                phi = -phi
        else:
            phi = 0

        #Calcular canvi de base
        vel_i_1 = np.linalg.norm(self.vel) * np.cos(theta) * u + np.linalg.norm(self.vel) * np.sin(theta) * w
        vel_i_2 = np.linalg.norm(other_ball.vel) * np.cos(phi) * u + np.linalg.norm(other_ball.vel) * np.sin(phi) * w

        vel_i_1_u = np.linalg.norm(self.vel) * np.cos(theta)
        vel_i_1_w = np.linalg.norm(self.vel) * np.sin(theta)

        vel_i_2_u = np.linalg.norm(other_ball.vel) * np.cos(phi)
        vel_i_2_w = np.linalg.norm(other_ball.vel) * np.sin(phi)

        #Calcular velocitats finals amb nova base
        x_1 = ((self.mass - other_ball.mass) / (self.mass + other_ball.mass)) * vel_i_1_u + ((2 * other_ball.mass)/(self.mass + other_ball.mass)) * vel_i_2_u
        x_2 =  ((2 * self.mass) / (self.mass + other_ball.mass)) * vel_i_1_u + ((other_ball.mass - self.mass) / (self.mass + other_ball.mass)) * vel_i_2_u
        
        self.vel = x_1 * u +  vel_i_1_w * w
        other_ball.vel = x_2 * u + vel_i_2_w * w

    def draw(self, screen):
        pygame.draw.circle(screen, self.color, self.pos, self.radius)

    def checkScreenEdges(self, screen:pygame.Surface):
        #Bottom edge
        if (self.pos[1] > screen.get_height() - self.radius):
            self.vel[1] = -self.vel[1]
            self.pos[1] = screen.get_height() - self.radius

        #Top edge
        if (self.pos[1] < 0 + self.radius):
            self.vel[1] = -self.vel[1]
            self.pos[1] = self.radius

        #Left edge
        if (self.pos[0] < 0 + self.radius):
            self.vel[0] = -self.vel[0]
            self.pos[0] = self.radius

        #Right edge
        if (self.pos[0] > screen.get_width() - self.radius):
            self.vel[0] = -self.vel[0]
            self.pos[0] = screen.get_width() - self.radius

    def apply_normal_and_friction_force(self, inclination, coef):
        N = -(self.gravity[1] * self.mass * np.cos(inclination))
        self.apply_force([-coef * N * -self.vel[0], -coef * N * -self.vel[1]])
        pass

# Constants
WIDTH = 800
HEIGHT = 600
RADI_ORBITAL = 15
COLOR_ORBITAL = (100, 150, 255)
COLOR_TEXT = (255, 255, 255)
NUM_POPULATION = 10
  
frames = 60
factor = 15
finish = False
# Pygame inicialització
screen = pygame.display.set_mode((WIDTH, HEIGHT))
pygame.display.set_caption("Visualització Matriu LS-28")
font = pygame.font.SysFont(None, 24)
clock = pygame.time.Clock()

#GAME Setup
# Títol
font = pygame.font.SysFont("Arial", 30)
ui_title = font.render('Agar.io', True, "white")

# Create food
food_1 = Food(10, 'red')
food_2 = Food(10, 'orange')
food_3 = Food(10, 'gray')
food_4 = Food(10, 'pink')
food_5 = Food(10, 'purple')
food_6 = Food(10, 'green')
food_7 = Food(10, 'turquoise')
food_8 = Food(10, 'black')

food = [food_1, food_2, food_3, food_4, food_5, food_6, food_7, food_8]

# Create ball with applied force
ball_1 = Ball(20, 10, np.array([500, 500]), 'white')
ball_2 = Ball(20, 10, np.array([100, 100]), 'yellow')


def draw_result(individual):
    screen.fill((20, 20, 30))
    marge_x = WIDTH // (28 + 1)
    marge_y = HEIGHT

    posicions = []
    y = marge_y / 2
    for i in range(28):
        x = (i + 1) * marge_x
        posicions.append((x, y))

   
    orbital_sums = np.sum(individual, axis=0)  

    for idx, (x, y) in enumerate(posicions):
        value = int(orbital_sums[idx])
        if value > 0:
            pygame.draw.circle(screen, COLOR_ORBITAL, (x, y), RADI_ORBITAL)
            text = font.render(str(value), True, COLOR_TEXT)
            text_rect = text.get_rect(center=(x, y))
            screen.blit(text, text_rect)
        else:
            pygame.draw.circle(screen, COLOR_ORBITAL, (x, y), RADI_ORBITAL, 2)

    pygame.display.update()

def execute_game():
    screen.fill((20, 20, 30))
    #BALL 1 LOGIC
    screen.blit(ui_title, (30, 30))

    #BALL 1 LOGIC
    ball_1.check_collision(screen, food, ball_2)
    ball_1.checkScreenEdges(screen)
    ball_1.update(factor/frames, food)

    #BALL 2 LOGIC
    #BALL 1 LOGIC
    ball_2.check_collision(screen, food, ball_1)
    ball_2.checkScreenEdges(screen)
    ball_2.update(factor/frames, food)

    ball_1.draw(screen)
    ball_2.draw(screen)

    for f in food:
        f.draw(screen)
    pygame.display.update()

#Inicialitzar la matriu
population = [create_adn() for _ in range(NUM_POPULATION)]
# Bucle principal
running = True
while running:
    clock.tick(15)
    
    for event in pygame.event.get():
        if event.type == pygame.QUIT or (event.type == pygame.KEYDOWN and event.key == pygame.K_ESCAPE):
            running = False

    
    # Dibuixar nova configuració
    if finish == False:
        fitness_values = [fitness(adn) for adn in population]

        parents = elitist_selection(population, fitness_values, elite_size=4)
        childrenPopulation = []
        while len(childrenPopulation) < NUM_POPULATION:
            parent1 = random.choice(parents)
            parent2 = random.choice(parents)
            child = crossover_5050(parent1, parent2)
            child = mutate(child, mutation_rate=0.01)
            if len(childrenPopulation) < NUM_POPULATION:
                childrenPopulation.append(child)

        population = childrenPopulation
        execute_game()

    for i in range(NUM_POPULATION):
        if fitness_values[i] == 0:
            finish = True
            print("Found a solution!")
            print(population[i])
            draw_result(population[i])
            
        
            #draw(population, fitness_values)
            
        
pygame.quit()


Found a solution!
[[0 0 0 1 1 0 1 0 0 0 0 1 0 0 0 1 1 0 0 0 0 0 1 1 0 1 0 0]
 [0 0 1 1 0 0 0 1 0 0 1 1 0 0 1 0 0 0 0 1 0 0 1 0 0 1 0 0]
 [0 0 1 1 0 1 0 0 1 1 0 1 0 0 0 0 1 1 0 1 0 1 0 0 1 1 0 0]
 [1 0 1 0 0 0 0 0 0 0 0 0 1 1 0 1 1 0 1 1 0 0 1 0 1 0 0 0]
 [0 0 0 0 0 1 0 1 0 1 0 0 0 1 0 0 0 1 1 0 0 1 0 0 0 0 0 1]
 [1 1 0 1 0 1 0 0 0 1 0 1 0 0 1 1 0 0 1 0 1 0 0 0 0 0 1 1]
 [0 0 1 1 0 1 0 0 0 0 1 0 1 0 0 1 0 1 1 0 0 1 0 0 0 0 1 1]
 [0 0 1 1 0 1 0 0 1 0 0 0 0 0 0 1 0 1 1 0 0 1 0 1 0 0 0 0]
 [1 0 0 0 1 0 0 0 0 0 0 0 1 0 1 0 0 1 0 0 0 0 0 0 0 0 0 0]
 [1 0 1 0 0 0 1 0 0 1 0 0 0 0 0 0 1 0 0 0 0 1 0 1 0 0 1 0]
 [1 0 0 0 0 0 1 0 0 1 0 0 1 0 1 1 0 0 1 0 1 0 1 1 0 0 0 0]
 [1 0 0 0 1 0 0 0 0 1 0 0 1 0 0 0 1 0 0 1 0 0 0 1 0 0 0 0]
 [0 1 0 0 0 0 0 1 0 1 1 0 0 1 0 0 0 1 0 0 1 0 1 1 0 1 0 0]
 [0 0 1 0 1 0 0 1 0 0 1 0 0 0 0 1 1 0 0 0 0 1 1 0 0 1 1 0]
 [0 1 0 0 1 0 0 0 0 0 0 1 1 0 1 0 0 1 0 0 0 0 1 1 0 1 0 1]
 [0 0 1 1 0 0 0 0 1 0 0 0 0 0 1 0 0 1 0 1 1 1 1 0 1 1 0 0]]
Found a solution!
[[0 0 0 1 1 0 1 0 0